# TEI Conference 2019 #
## Préambule ##

Avant de proposer une analyse numérique des thèmes et sujets des Conférences TEI de l'année 2019, nous allons dans un premier temps télécharger toutes les données depuis le site web dédié, les trier et les stocker en local.

Ensuite nous normaliserons les textes, pour enfin les analyser.



## Etape 1 : Le Téléchargement ##

### Les packages ###

Exécutez la cellule ci-dessous une seule fois, au premier lancement.

In [1]:
#Les packages à installer d'abord. Il faut le faire une seule fois, et très certainement redémarrer le noyau.
pip install bs4
pip install lxml
pip install tqdm

SyntaxError: invalid syntax (<ipython-input-1-34475629646c>, line 2)

Il faut lancer la cellule ci-dessous à chaque fois.

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import json
import os
from tqdm import tqdm
#Créer un certificat SSL pour sécuriser l'accès au site internet.
import ssl 

ssl._create_default_https_context = ssl._create_unverified_context 

### La récupération des liens internes ###

Le site de la Conférence TEI 2019 a stocké les abstracts d'une manière particulière : Une page web renvoie grâce à des liens internes aux différents abstracts encodés en TEI. Ainsi, il faut grâce à la librairie BeautifulSoup parser le code HTML de la page, relever tous les liens internes vers les abstracts en XML, leur adjoindre l'adresse du site web pour obtenir l'URL et les stocker ensuite dans un document JSON en cache. 

In [6]:
content = urlopen('https://gams.uni-graz.at/context:tei2019.papers?context=context:tei2019.papers')
HTML = content.read()

soup = BeautifulSoup(HTML, 'html.parser') # Je parse le code HTML de la page web ci-dessus pour pouvoir le traiter avec Python
listLink = [] # J'instancie une liste dans laquelle j'ajouterai tous les liens internes.
print("Saisie des liens")
for link in tqdm(soup.find_all('a')): # Je cherche dans le code HTML toutes les balises <a>, et cherche la valeur des
    # attributs href, notamment en les triant sur la base d'un motif présent dans tous les liens internes concernés.
    if 'TEI_SOURCE' in link.get('href'):
        lien = link.get('href')
        listLink.append(lien)

# Je crée le dossier cache.
if not os.path.exists("./cache2019"):
    os.makedirs("./cache2019")
if not os.path.exists("./cache2019/cacheXML"):
    os.makedirs("./cache2019/cacheXML")

 #Dans le cache, je crée un fichier JSON dans lequel je stocke les URL des abstracts.
with open('./cache2019/liste_des_liens_internet.json', 'w') as liste_des_liens_internet:
    i = 0 # Je crée un compteur qui servira de clés 
    liste_lien = {} #Je stocke les URL dans un dictionnaire que je vais dump ensuite directement dans le JSON
    print("Stockage des liens")
    for linky in tqdm(listLink):
        lien = 'https://gams.uni-graz.at' + linky #Je rajoute l'adresse du site au lien interne
        liste_lien[i] = lien
        i += 1
    json.dump(liste_lien, liste_des_liens_internet)


100%|██████████| 46/46 [00:00<00:00, 152761.67it/s]

Saisie des liens
Stockage des liens


### La récupération des contenus ###

Ensuite, on utilise la liste des URL stockées dans le JSON pour télécharger les abstracts dans un dictionnaire qui a en clé un entier.

In [7]:
i = 0
with open('./cache2019/liste_des_liens_internet.json', 'r') as liste_des_liens_internet:
    dicoXML = {}
    liste_liens_lisible = liste_des_liens_internet.read()
    dico_liens = json.loads(liste_liens_lisible)
    print("Téléchargement en cours...")
    for link in tqdm(dico_liens):
        lien = dico_liens[link]
        XML = urlopen(lien).read()
        XML = XML.decode("UTF-8")
        dicoXML[i] = XML
        print("téléchargement accompli : {0}/46".format(i))
        i += 1
    print("Téléchargement terminé.")


  0%|          | 0/46 [00:00<?, ?it/s]

Téléchargement en cours...


  2%|▏         | 1/46 [00:00<00:42,  1.06it/s]

téléchargement accompli : 0/46


  4%|▍         | 2/46 [00:01<00:40,  1.08it/s]

...

téléchargement accompli : 43/46


 98%|█████████▊| 45/46 [00:45<00:00,  1.06it/s]

téléchargement accompli : 44/46


100%|██████████| 46/46 [00:46<00:00,  1.01s/it]

téléchargement accompli : 45/46
Téléchargement terminé.


### Extraction de données ###

Dans cette dernière partie, je vais d'abord extraire les noms des auteurs, les titres des abstracts, les mots-clés donnés et les institutions d'origine des auteurs et les stocker dans un fichier csv. J'utilise notamment la librairie BeautifulSoup pour parser les abstracts en XML.

L'objectif pour remplir le fichier csv va être de constituer des listes d'un seul élément (pour chaque case : un élément regroupant tous les noms, un autre regroupant toutes institutions, etc...) et où l'élément à l'intérieur doit être une str, car c'est un format convenable pour remplir du csv.

In [8]:
with open('./cache2019/TEI2019.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    i = 0
    for key in dicoXML:
        value = dicoXML[i]
        soup = BeautifulSoup(value, 'xml')
        print("Récupération des autorités...")
        for contributeur in tqdm(soup.find_all('author')): #Je récupère tous les noeuds "author"
            for elem in soup.name: #Je vais récupérer ici les éléments de prénom et de nom de chaque auteur
                liste_noms_balise = soup.find_all(['forename', 'surname'])
                noms_a_rassembler = [] #j'instancie une liste dans lequel je vais ranger chaque élément de nom ou de prénom de tous les auteurs d'un seul abstract
                for nom in liste_noms_balise: #ici, je vais normaliser en les transformant en str et en retirant les balises manuellement.
                    nom = str(nom)
                    nom = nom.replace("<forename>", "")
                    nom = nom.replace('<forename full="yes">', "")
                    nom = nom.replace("</forename>", "")
                    nom = nom.replace("</surname>", "")
                    nom = nom.replace("<surname>", "")
                    nom = nom.replace('<surname full="yes">', "")
                    nom = nom.replace('<surname  full="init">', "")
                    nom = nom.replace('<forename full="init">', "")
                    nom = nom.replace('<forename full="abb">', "")
                    noms_a_rassembler.append(nom)
            for elem in soup.publisher: #Ici je récupere directement une str grâce à la méthode .contents.
                organisation = soup.orgName.contents
                orga = organisation[0].split() #Pour éliminer les sauts de lignes, tab et autres espaces gênants, la solution la plus simple fut d'enchaîner un .split et un .join
                orga = ' '.join(orga)
                orga = [orga]
            for elem in soup.titleStmt:
                titre = soup.title.contents #grâce à la méthode .contents, je récupère directement une str, sous condition qu'il n'y ait pas de balises imbriquées (comme pour les noms)
                if len(titre) > 1:
                    titre = titre[0]
                    titre = [titre]
        print("Récupération des mots-clés...")
        for elem in tqdm(soup.keywords):
            liste_motcles = soup.find_all('term') #je récupère tous les mots clés dans une liste
            retypage_liste = [str(i) for i in liste_motcles] #je transforme le type des éléments de la liste en str chacun
            motcles = str(" | ".join(retypage_liste)) #je transforme la liste en str
            motcles = motcles.replace("<term>", "") #je nettoie la str
            motcles = motcles.replace("</term>", "")
            motcles = [motcles] #je passe cette grande str en une liste avec 1 seul élément
        abstract = soup("body") #ici, j'enregistre le contenu de l'abstract, et je ne garde que le premier élément, car les autres éléments capturés sont trop compliqués à traiter, et trop peu intéressants.
        abstract = abstract[0]
        titre_dans_cache = titre[0] #je nomme les abstracts d'après leur titre, cela est utile dans l'étape 2.
        titre_dans_cache = titre_dans_cache.replace(" ", "_") #je supprime les espaces dans les noms de fichier, car ça gêne Python
        with open('./cache2019/cacheXML/%s.xml' % titre_dans_cache, 'w', encoding='UTF-8') as docXML:
            docXML.write(str(abstract))
        titre_dans_cache = titre_dans_cache + ".xml"
        noms_rassembles = ' '.join(noms_a_rassembler) #tous les noms saisis plus haut, chacun étant un élément d'une grande liste, sont rassemblés dans une str, qui est ensuite transformée en une liste d'un seul élément
        noms = [noms_rassembles]
        masterList = noms + orga + titre + motcles #j'assemble chacune des listes composées d'un seul élément en une nested liste. Logiquement, cela ressemble à : [[str], [str], [str], [str]]
        writer.writerow(masterList)
        i += 1 #ceci est le compteur permettant de trouver les clés du dictionnaire, où la valeur est l'abstract en XML


 36%|███▋      | 4/11 [00:00<00:00, 39.41it/s]

Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...


 50%|█████     | 2/4 [00:00<00:00, 18.96it/s]

Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...


  0%|          | 0/10 [00:00<?, ?it/s]

...

Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...


 16%|█▌        | 3/19 [00:00<00:00, 26.85it/s]

Récupération des autorités...


100%|██████████| 9/9 [00:00<00:00, 3151.51it/s]

Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
Récupération des autorités...
Récupération des mots-clés...
